

```
Followed instructions from https://github.com/tensorflow/models/blob/master/research/deeplab/g3doc/pascal.md

Small fixes were found in: https://github.com/RomRoc/deeplab_retrain_ade20k_colab/blob/master/deeplab_retrain_ade20k.ipynb
```



In [1]:
#from google.colab import drive
#drive.mount('/content/gdrive')
#%cd /content/gdrive

# Tensorflow 1.15 setup with models (a restart might be needed after installation)

In [2]:
#UNCOMMENT THESE ONLY IF ON GOOGLE COLAB

#!pip uninstall -y tensorflow
#!pip install tensorflow==1.15.0 tf_slim 
#%env PYTHONPATH=:/env/python/:/content/models/research/:/content/models/research/slim:


In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
print(tf.__version__)

#Uncomment the bellow for NVIDIA local issues.
tf.config.experimental_run_functions_eagerly(True)

1.15.0


In [4]:
rootdir=os.getcwd()
print("Working environment: ",rootdir)
if not os.path.exists("models"):
  !git clone https://github.com/tensorflow/models.git
else:
  print("Tensorflow models already cloned. Right?")
%env PYTHONPATH=/env/python/:{rootdir}/models/research/:{rootdir}/models/research/slim
%cd models/research/deeplab



Working environment:  /mnt/ext4/mauricio/nas_deeplearning
Tensorflow models already cloned. Right?
env: PYTHONPATH=/env/python/:/mnt/ext4/mauricio/nas_deeplearning/models/research/:/mnt/ext4/mauricio/nas_deeplearning/models/research/slim
/mnt/ext4/mauricio/nas_deeplearning/models/research/deeplab


In [5]:
#Uncommend and run this if you would like to test your environmnent
!python model_test.py


Running tests under Python 3.7.10: /mnt/ext4/mauricio/nas_deeplearning/autodeeplab/bin/python
[ RUN      ] DeeplabModelTest.testBuildDeepLabWithDensePredictionCell
Instructions for updating:
Use `self.session()` or `self.cached_session()` instead.
W0526 21:26:04.159153 139908479231808 deprecation.py:323] From /mnt/ext4/mauricio/nas_deeplearning/autodeeplab/lib/python3.7/contextlib.py:82: TensorFlowTestCase.test_session (from tensorflow.python.framework.test_util) is deprecated and will be removed in a future version.
Instructions for updating:
Use `self.session()` or `self.cached_session()` instead.

W0526 21:26:04.271929 139908479231808 module_wrapper.py:139] From model_test.py:134: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

/mnt/ext4/mauricio/nas_deeplearning/autodeeplab/lib/python3.7/site-packages/tensorflow_core/python/framework/tensor_util.py:521: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_conte

In [6]:
rootdir

'/mnt/ext4/mauricio/nas_deeplearning'

# Dataset

In [7]:
#Downloading the dataset
# From the tensorflow/models/research/deeplab/datasets directory.
%cd {rootdir}/models/research/deeplab/datasets
datasetdir=os.path.join(os.getcwd(),"pascal_voc_seg")
if not os.path.exists(datasetdir) or not os.path.exists(os.path.join(datasetdir,"VOC2012")):
  !sh download_and_convert_voc2012.sh
else:
  print("Dataset already downloaded. Right?")

/mnt/ext4/mauricio/nas_deeplearning/models/research/deeplab/datasets
Dataset already downloaded. Right?


# Download pretrained weights

In [8]:
%cd {rootdir}/models/research/deeplab/datasets
if not os.path.exists('deeplabv3_xception_2018_01_04.tar.gz'):
  !wget http://download.tensorflow.org/models/deeplabv3_xception_2018_01_04.tar.gz
!tar -xzvf  'deeplabv3_xception_2018_01_04.tar.gz' 
initial_checkpoint_dir=os.path.join(os.getcwd(),'xception')

/mnt/ext4/mauricio/nas_deeplearning/models/research/deeplab/datasets
xception/
xception/model.ckpt.index
xception/model.ckpt.data-00000-of-00001


In [9]:
initial_checkpoint_dir


'/mnt/ext4/mauricio/nas_deeplearning/models/research/deeplab/datasets/xception'

# Training

In [10]:
#If this cell ran without errors, DeepLab's shell command might work.
!python -c "import tensorflow.contrib"

In [11]:
PATH_TO_INITIAL_CHECKPOINT=f"{initial_checkpoint_dir}/model.ckpt.index"
PATH_TO_TRAIN_DIR=datasetdir
PATH_TO_DATASET=os.path.join(datasetdir,'tfrecord')
%cd {rootdir}/models/research/deeplab/

/mnt/ext4/mauricio/nas_deeplearning/models/research/deeplab


In [12]:
#https://github.com/tensorflow/models/issues/7201
!export TF_FORCE_GPU_ALLOW_GROWTH=true
!python train.py --logtostderr --training_number_of_steps=100 --train_split="train" --model_variant="nas_hnasnet" \
  --atrous_rates=12 \
  --atrous_rates=24 \
  --atrous_rates=36 \
  --output_stride=8 \
  --decoder_output_stride=4 \
  --train_crop_size=513,513 \
  --train_batch_size=4 \
  --learning_rate=.003 \
  --dataset="pascal_voc_seg" --train_logdir={PATH_TO_TRAIN_DIR} --dataset_dir={PATH_TO_DATASET}
  


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0526 21:26:17.535015 140592949385024 module_wrapper.py:139] From train.py:274: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0526 21:26:17.535103 140592949385024 module_wrapper.py:139] From train.py:274: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0526 21:26:17.535189 140592949385024 module_wrapper.py:139] From train.py:289: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0526 21:26:17.535265 140592949385024 module_wrapper.py:139] From train.py:290: The name tf.logging.info is deprec


W0526 21:26:25.053512 140592949385024 module_wrapper.py:139] From /mnt/ext4/mauricio/nas_deeplearning/models/research/deeplab/utils/train_utils.py:158: The name tf.losses.add_loss is deprecated. Please use tf.compat.v1.losses.add_loss instead.


W0526 21:26:25.053698 140592949385024 module_wrapper.py:139] From train.py:326: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.


W0526 21:26:25.053754 140592949385024 module_wrapper.py:139] From train.py:326: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.


W0526 21:26:25.054414 140592949385024 module_wrapper.py:139] From train.py:332: The name tf.model_variables is deprecated. Please use tf.compat.v1.model_variables instead.


W0526 21:26:25.054482 140592949385024 module_wrapper.py:139] From train.py:333: The name tf.summary.histogram is deprecated. Please use tf.compat.v1.summary.histogram instead.


W0526 21:26:25.697617 140592949385024 module_wrapper.py:139] From t

INFO:tensorflow:global step 70: loss = 2.5447 (4.198 sec/step)
I0526 21:31:55.468215 140592949385024 learning.py:507] global step 70: loss = 2.5447 (4.198 sec/step)
INFO:tensorflow:global step 80: loss = 2.6204 (4.059 sec/step)
I0526 21:32:37.011996 140592949385024 learning.py:507] global step 80: loss = 2.6204 (4.059 sec/step)
INFO:tensorflow:global step 90: loss = 2.6915 (4.095 sec/step)
I0526 21:33:18.882760 140592949385024 learning.py:507] global step 90: loss = 2.6915 (4.095 sec/step)
INFO:tensorflow:global step 100: loss = 2.5539 (4.344 sec/step)
I0526 21:34:00.864470 140592949385024 learning.py:507] global step 100: loss = 2.5539 (4.344 sec/step)
INFO:tensorflow:Stopping Training.
I0526 21:34:00.864780 140592949385024 learning.py:777] Stopping Training.
INFO:tensorflow:Finished training! Saving model to disk.
I0526 21:34:00.864886 140592949385024 learning.py:785] Finished training! Saving model to disk.
/mnt/ext4/mauricio/nas_deeplearning/autodeeplab/lib/python3.7/site-packages/

# Evaluation

In [13]:
#!ls {PATH_TO_DATASET}
!echo {PATH_TO_TRAIN_DIR}

/mnt/ext4/mauricio/nas_deeplearning/models/research/deeplab/datasets/pascal_voc_seg


In [ ]:
# From tensorflow/models/research/
!python eval.py \
    --logtostderr \
    --eval_split="val" \
    --model_variant="nas_hnasnet" \
    --atrous_rates=12 \
    --atrous_rates=24 \
    --atrous_rates=36 \
    --output_stride=8 \
    --decoder_output_stride=4 \
    --eval_crop_size="513,513" \
    --dataset="pascal_voc_seg" \
    --checkpoint_dir={PATH_TO_TRAIN_DIR} \
    --eval_logdir={PATH_TO_TRAIN_DIR} \
    --dataset_dir={rootdir}/models/research/deeplab/datasets/pascal_voc_seg/tfrecord
#{PATH_TO_DATASET}




W0526 21:34:28.381084 140223054567232 module_wrapper.py:139] From eval.py:91: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0526 21:34:28.381266 140223054567232 module_wrapper.py:139] From eval.py:91: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0526 21:34:28.381455 140223054567232 module_wrapper.py:139] From eval.py:108: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0526 21:34:28.381594 140223054567232 module_wrapper.py:139] From eval.py:109: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:Evaluating on val set
I0526 21:34:28.381656 140223054567232 eval.py:109] Evaluating on val set

W0526 21:34:28.382411 140223054567232 module_wrapper.py:139] From /mnt/ext4/mauricio/nas_deeplearning/models/research/deeplab/datasets/data_generator.py:350: The name tf.gfile.Glob is deprecated. Please us

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
W0526 21:34:34.297644 140223054567232 deprecation.py:323] From /mnt/ext4/mauricio/nas_deeplearning/autodeeplab/lib/python3.7/site-packages/tensorflow_core/python/ops/metrics_impl.py:1178: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.

W0526 21:34:34.301754 140223054567232 module_wrapper.py:139] From eval.py:171: The name tf.metrics.true_positives is deprecated. Please use tf.compat.v1.metrics.true_positives instead.


W0526 21:34:34.308858 140223054567232 module_wrapper.py:139] From eval.py:174: The name tf.metrics.false_positives is deprecated. Please use tf.compat.v1.metrics.false_positives instead.


W0526 21:34:34.316140 140223054567232 module_wrapper.py:139] From eval.py:177: The name tf.metrics.false_negatives is deprecated. Please use tf.compat.v1.metrics.false_negatives inst

67 ops no flops stats due to incomplete shapes.
Parsing Inputs...

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. N

INFO:tensorflow:Waiting for new checkpoint at /mnt/ext4/mauricio/nas_deeplearning/models/research/deeplab/datasets/pascal_voc_seg
I0526 21:34:36.606898 140223054567232 evaluation.py:189] Waiting for new checkpoint at /mnt/ext4/mauricio/nas_deeplearning/models/research/deeplab/datasets/pascal_voc_seg
INFO:tensorflow:Found new checkpoint at /mnt/ext4/mauricio/nas_deeplearning/models/research/deeplab/datasets/pascal_voc_seg/model.ckpt-100
I0526 21:34:36.607975 140223054567232 evaluation.py:198] Found new checkpoint at /mnt/ext4/mauricio/nas_deeplearning/models/research/deeplab/datasets/pascal_voc_seg/model.ckpt-100
INFO:tensorflow:Graph was finalized.
I0526 21:34:37.942645 140223054567232 monitored_session.py:240] Graph was finalized.
INFO:tensorflow:Restoring parameters from /mnt/ext4/mauricio/nas_deeplearning/models/research/deeplab/datasets/pascal_voc_seg/model.ckpt-100
I0526 21:34:38.205827 140223054567232 saver.py:1284] Restoring parameters from /mnt/ext4/mauricio/nas_deeplearning/mo

In [ ]:
EXPORT_PATH="/content/models/export/"
if not os.path.exists(EXPORT_PATH): os.makedirs(EXPORT_PATH)
CKPT_PATH="/content/models/research/deeplab/datasets/pascal_voc_seg/model.ckpt-50"

In [ ]:
model_name="frozen_inference_graph.pb"
print('START export_model.py')
!python export_model.py \
  --logtostderr \
  --checkpoint_path="{CKPT_PATH}" \
  --export_path="{EXPORT_PATH}/{model_name}" \
  --model_variant="nas_hnasnet" \
  --inference_scales=1.0 \
  --atrous_rates=12 \
  --atrous_rates=24 \
  --atrous_rates=36 \
  --output_stride=8 \
  --decoder_output_stride=4 \
  --train_crop_size=513,513 \
  --train_batch_size=4 \
  --num_classes=21 \


# Deeplab's helper class

In [ ]:
#Copyed from DeepLab Demo
import os
from io import BytesIO
import tarfile
import tempfile
from six.moves import urllib

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

class DeepLabModel(object):
  """Class to load deeplab model and run inference."""

  INPUT_TENSOR_NAME = 'ImageTensor:0'
  OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
  INPUT_SIZE = 513
  FROZEN_GRAPH_NAME = 'frozen_inference_graph'

  def __init__(self, tarball_path):
    """Creates and loads pretrained deeplab model."""
    self.graph = tf.Graph()

    graph_def = None
    # Extract frozen graph from tar archive.
    tar_file = tarfile.open(tarball_path)
    for tar_info in tar_file.getmembers():
      if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
        file_handle = tar_file.extractfile(tar_info)
        graph_def = tf.GraphDef.FromString(file_handle.read())
        break

    tar_file.close()

    if graph_def is None:
      raise RuntimeError('Cannot find inference graph in tar archive.')

    with self.graph.as_default():
      tf.import_graph_def(graph_def, name='')

    self.sess = tf.Session(graph=self.graph)

  def run(self, image):
    """Runs inference on a single image.

    Args:
      image: A PIL.Image object, raw input image.

    Returns:
      resized_image: RGB image resized from original input image.
      seg_map: Segmentation map of `resized_image`.
    """
    width, height = image.size
    resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
    batch_seg_map = self.sess.run(
        self.OUTPUT_TENSOR_NAME,
        feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
    seg_map = batch_seg_map[0]
    return resized_image, seg_map


def create_pascal_label_colormap():
  """Creates a label colormap used in PASCAL VOC segmentation benchmark.

  Returns:
    A Colormap for visualizing segmentation results.
  """
  colormap = np.zeros((256, 3), dtype=int)
  ind = np.arange(256, dtype=int)

  for shift in reversed(range(8)):
    for channel in range(3):
      colormap[:, channel] |= ((ind >> channel) & 1) << shift
    ind >>= 3

  return colormap


def label_to_color_image(label):
  """Adds color defined by the dataset colormap to the label.

  Args:
    label: A 2D array with integer type, storing the segmentation label.

  Returns:
    result: A 2D array with floating type. The element of the array
      is the color indexed by the corresponding element in the input label
      to the PASCAL color map.

  Raises:
    ValueError: If label is not of rank 2 or its value is larger than color
      map maximum entry.
  """
  if label.ndim != 2:
    raise ValueError('Expect 2-D input label')

  colormap = create_pascal_label_colormap()

  if np.max(label) >= len(colormap):
    raise ValueError('label value too large.')

  return colormap[label]


def vis_segmentation(image, seg_map):
  """Visualizes input image, segmentation map and overlay view."""
  plt.figure(figsize=(15, 5))
  grid_spec = gridspec.GridSpec(1, 4, width_ratios=[6, 6, 6, 1])

  plt.subplot(grid_spec[0])
  plt.imshow(image)
  plt.axis('off')
  plt.title('input image')

  plt.subplot(grid_spec[1])
  seg_image = label_to_color_image(seg_map).astype(np.uint8)
  plt.imshow(seg_image)
  plt.axis('off')
  plt.title('segmentation map')

  plt.subplot(grid_spec[2])
  plt.imshow(image)
  plt.imshow(seg_image, alpha=0.7)
  plt.axis('off')
  plt.title('segmentation overlay')

  unique_labels = np.unique(seg_map)
  ax = plt.subplot(grid_spec[3])
  plt.imshow(
      FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
  ax.yaxis.tick_right()
  plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
  plt.xticks([], [])
  ax.tick_params(width=0.0)
  plt.grid('off')
  plt.show()


LABEL_NAMES = np.asarray([
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
])

FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)

# Checking the generated model

In [ ]:
!pwd

In [ ]:
print(f"Loading model from {EXPORT_PATH}{model_name}")
import tarfile
with tarfile.open('deeplab_model.tar.gz', 'w:gz') as tar:
  tar.add( os.path.join(EXPORT_PATH,model_name), arcname=model_name)
MODEL = DeepLabModel(os.path.join(os.getcwd(),'deeplab_model.tar.gz'))
print('Model loaded.')


In [ ]:
dir(MODEL)


In [ ]:
from google.colab import files
from os import path

file_name = 'img.jpg'
uploaded = files.upload()
  
for name, data in uploaded.items():
  with open(file_name, 'wb') as f:
    f.write(data)
    f.close()
    print('saved file ' + name)

orignal_im = Image.open(name)

print(f'running deeplab on image {file_name}...') 
resized_im, seg_map = MODEL.run(orignal_im)
vis_segmentation(resized_im, seg_map)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir mylogdir

# Custom data

In [ ]:
!python datasets/build_voc2012_data.py \
  --image_folder="/content/models/research/deeplab/datasets/PQR/dataset/JPEGImages" \
  --semantic_segmentation_folder="/content/models/research/deeplab/datasets/PQR/dataset/SegmentationClassRaw" \
  --list_folder="/content/models/research/deeplab/datasets/PQR/dataset/ImageSets" \
  --image_format="jpg" \
  --output_dir="/content/models/research/deeplab/datasets/PQR/tfrecord"